# Imports

In [103]:
from cmd import Cmd
from glob import glob
import importlib.resources
from io import StringIO
import json
import os
from os import chdir, curdir, listdir
from pathlib import Path
import pkg_resources
from pprint import pprint as pp
import re
import requests
import site
import sys
import zipfile

In [104]:
from bs4 import BeautifulSoup
from pandas import DataFrame, Series
from walkdir import filtered_walk

In [3]:
site.addsitedir(Path.home() / 'Development/hw-4.2.0/hw')

In [105]:
help(filtered_walk)

Help on function filtered_walk in module walkdir:

filtered_walk(top, included_files=None, included_dirs=None, excluded_files=None, excluded_dirs=None, depth=None, followlinks=False, min_depth=None)
    This is a wrapper around ``os.walk()`` and other filesystem traversal
    iterators, with these additional features:
    
     - *top* may be either a string (which will be passed to ``os.walk()``)
       or any iterable that produces sequences with ``path, subdirs, files``
       as the first three elements in the sequence
     - allows independent glob-style filters for filenames and subdirectories
     - allows a recursion depth limit to be specified
     - allows a minimum depth to be specified to report only subdirectory
       contents
     - emits a message to stderr and skips the directory if a symlink loop
       is encountered when following links
    
    Filtered walks created by passing in a string are always top down, as
    the subdirectory listings must be altered to pro

In [4]:
from constants import *
from tools import *

In [5]:
# [str(r) for r in pkg_resources.working_set.by_key['hw'].requires()]

In [6]:
sys.path

['/home/fuzzy/Projects/develop-4.2.0/hw-4.2.0/nb',
 '/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '',
 '/home/fuzzy/.local/lib/python3.8/site-packages',
 '/usr/local/lib/python3.8/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/home/fuzzy/.local/lib/python3.8/site-packages/IPython/extensions',
 '/home/fuzzy/.ipython',
 '/home/fuzzy/Development/hw-4.2.0/hw']

## Get a List of Python Modules

In [7]:
try:
    src = Path("/usr/share/doc/python3.7-doc/html/py-modindex.html").read_text()
except:
    try:
        src = requests.get("https://docs.python.org/3/py-modindex.html").text
    except:
        pass

class BS(BeautifulSoup):
    def __init__(self, s):
        super().__init__(s, features="html.parser")

PYTHON_MODULES = None
try:
    PYTHON_MODULES = {a.text.split('.')[0] for a in BS(src).findAll("a") if len(a.text) > 1 and not a.text[0].isupper()}
except:
    path = "/usr/lib/python3.8"
    PYTHON_MODULES = list(sorted([n for n in listdir(path) if not n.startswith(PERIOD)], key=str.lower))


In [109]:
DataFrame(list(filtered_walk(site.USER_SITE, included_files=['*.py'])))

,0,1,2
0,/home/fuzzy/.local/lib/python3.8/site-packages,"[jsonschema, nbclient, jupyterlab_widgets, Sen...","[walkdir.py, jupyter.py, mistune.py, nest_asyn..."
1,/home/fuzzy/.local/lib/python3.8/site-packages...,"[tests, benchmarks, schemas, __pycache__]","[validators.py, _reflect.py, __main__.py, _typ..."
2,/home/fuzzy/.local/lib/python3.8/site-packages...,[__pycache__],"[_helpers.py, test_validators.py, test_jsonsch..."
3,/home/fuzzy/.local/lib/python3.8/site-packages...,[],[]
4,/home/fuzzy/.local/lib/python3.8/site-packages...,[__pycache__],"[json_schema_test_suite.py, __init__.py, issue..."
...,...,...,...
1716,/home/fuzzy/.local/lib/python3.8/site-packages...,[fr_FR],[]
1717,/home/fuzzy/.local/lib/python3.8/site-packages...,[LC_MESSAGES],[]
1718,/home/fuzzy/.local/lib/python3.8/site-packages...,[],[]
1719,/home/fuzzy/.local/lib/python3.8/site-packages...,[],[]


In [8]:
PYTHON_MODULES

{'3',
 '__future__',
 '__main__',
 '_thread',
 'abc',
 'aifc',
 'argparse',
 'array',
 'ast',
 'asynchat',
 'asyncio',
 'asyncore',
 'atexit',
 'audioop',
 'base64',
 'bdb',
 'binascii',
 'binhex',
 'bisect',
 'builtins',
 'bz2',
 'cProfile',
 'calendar',
 'cgi',
 'cgitb',
 'chunk',
 'cmath',
 'cmd',
 'code',
 'codecs',
 'codeop',
 'collections',
 'colorsys',
 'compileall',
 'concurrent',
 'configparser',
 'contextlib',
 'contextvars',
 'copy',
 'copyreg',
 'crypt',
 'csv',
 'ctypes',
 'curses',
 'dataclasses',
 'datetime',
 'dbm',
 'decimal',
 'difflib',
 'dis',
 'distutils',
 'doctest',
 'email',
 'encodings',
 'ensurepip',
 'enum',
 'errno',
 'faulthandler',
 'fcntl',
 'filecmp',
 'fileinput',
 'fnmatch',
 'formatter',
 'fractions',
 'ftplib',
 'functools',
 'gc',
 'getopt',
 'getpass',
 'gettext',
 'glob',
 'graphlib',
 'grp',
 'gzip',
 'hashlib',
 'heapq',
 'hmac',
 'html',
 'http',
 'imaplib',
 'imghdr',
 'imp',
 'importlib',
 'index',
 'inspect',
 'io',
 'ipaddress',
 'itertools

## Examine the Contents of a `.zip` File

In [9]:
import zipfile

## Make a List of Python Distribution Modules

In [10]:
path = Series(filter(lambda p: p.exists(),               # drop paths that don't exist
                     filter(lambda s: bool(len(s.name)), # drop the empty `str`s
                            map(lambda s: Path(s),       # convert `str` to `Path`
                                sys.path
                               ))))

In [11]:
path = Series([p for p in
               [Path(s) for s in
                [s for s in sys.path if s]
               ] if p.exists()
              ])

In [12]:
path

0       /home/fuzzy/Projects/develop-4.2.0/hw-4.2.0/nb
1                                   /usr/lib/python3.8
2                       /usr/lib/python3.8/lib-dynload
3       /home/fuzzy/.local/lib/python3.8/site-packages
4               /usr/local/lib/python3.8/dist-packages
5                       /usr/lib/python3/dist-packages
6    /home/fuzzy/.local/lib/python3.8/site-packages...
7                                 /home/fuzzy/.ipython
8                  /home/fuzzy/Development/hw-4.2.0/hw
dtype: object

In [13]:
print(Path.cwd())
list(os.walk('.'))

/home/fuzzy/Projects/develop-4.2.0/hw-4.2.0/nb


[('.',
  ['.ipynb_checkpoints'],
  ['program.ipynb',
   'sftp.ipynb',
   'args.ipynb',
   'links.ipynb',
   'TODO.ipynb',
   'hw.ipynb',
   'sass.ipynb',
   'profiler.ipynb',
   'bash.ipynb',
   'pdf.ipynb',
   'xdg.ipynb',
   'lint.ipynb',
   'git.ipynb',
   'worksheet.ipynb',
   'library.ipynb',
   'hw.c.ipynb',
   'setup.ipynb',
   'startup.ipynb',
   'docs.ipynb',
   'run.ipynb',
   'links.md',
   'cgi.ipynb',
   'debug.ipynb',
   'driver.ipynb',
   'index.ipynb',
   'logs.ipynb',
   'tests.ipynb',
   'jqueryui.ipynb',
   'misc.ipynb',
   'imports.ipynb',
   'mariadb.ipynb',
   'less.ipynb',
   'create_project.ipynb',
   'tools.ipynb',
   'env.ipynb',
   'files.ipynb',
   'javasript.ipynb',
   'jquery.ipynb',
   'config.ipynb',
   'py2nb.ipynb',
   'python.ipynb',
   'linux.ipynb']),
 ('./.ipynb_checkpoints',
  [],
  ['lint-checkpoint.ipynb',
   'TODO-checkpoint.ipynb',
   'hw-checkpoint.ipynb',
   'jquery-checkpoint.ipynb',
   'create_project-checkpoint.ipynb',
   'git-checkpoint.

In [14]:
def get_mod_names(f=curdir, output=False):
    sio = StringIO()
    for w in os.walk(f):
        if w[0] == '.' or not Path(w[0]).stem[0] in {'.'}:
            for s in w[2]:
                if s.endswith('.py') and not (s.startswith('.')):
                    print((w[0].replace('/', '.').lstrip('.') + '.' + s).lstrip('.')[:-3], file=sio)
    s = Series([t for t in sio.getvalue().split('\n') if t], dtype=object)
    if output: print(s)
    return s

In [15]:
get_mod_names(output=True)

Series([], dtype: object)


Series([], dtype: object)

In [16]:
def get_sys_path_mods(output=False):
    mods = { 'python' : set(), 'dist' : set(), 'local' : set() }
    path = Series([p for p in
                   [Path(s) for s in
                    [s for s in sys.path if s]
                   ] if p.exists()
                  ])
    print(path)
    for p in path:
        n = get_mod_names(p)
        

In [17]:
get_sys_path_mods()

0       /home/fuzzy/Projects/develop-4.2.0/hw-4.2.0/nb
1                                   /usr/lib/python3.8
2                       /usr/lib/python3.8/lib-dynload
3       /home/fuzzy/.local/lib/python3.8/site-packages
4               /usr/local/lib/python3.8/dist-packages
5                       /usr/lib/python3/dist-packages
6    /home/fuzzy/.local/lib/python3.8/site-packages...
7                                 /home/fuzzy/.ipython
8                  /home/fuzzy/Development/hw-4.2.0/hw
dtype: object


In [18]:
columnize(sorted(listdir(path[6])))

__init__.py  autoreload.py   rmagic.py      sympyprinting.py
__pycache__  cythonmagic.py  storemagic.py  tests           


In [19]:
from pandas import DataFrame
DataFrame(os.walk('.'))

,0,1,2
0,.,[.ipynb_checkpoints],"[program.ipynb, sftp.ipynb, args.ipynb, links...."
1,./.ipynb_checkpoints,[],"[lint-checkpoint.ipynb, TODO-checkpoint.ipynb,..."


In [20]:
try:
    src = requests.get("https://docs.python.org/3/py-modindex.html").text
except:
    src = Path("/usr/share/doc/python3.7-doc/html").read_text()

class BS(BeautifulSoup):
    def __init__(self, s):
        super().__init__(s, features="html.parser")

PYTHON_MODULES = None
try:
    PYTHON_MODULES = {a.text.split('.')[0] for a in BS(src).findAll("a") if len(a.text) > 1 and not a.text[0].isupper()}
except:
    path = site.USER_SITE
    PYTHON_MODULES = list(sorted([n for n in listdir(path) if not n.startswith(PERIOD)], key=str.lower))

PACKAGE_MODULES = sorted([re.sub(".py$", "", n) for n in listdir(site.USER_SITE) if not re.search(".*-info", n)])


In [21]:
help(os.walk)

Help on function walk in module os:

walk(top, topdown=True, onerror=None, followlinks=False)
    Directory tree generator.
    
    For each directory in the directory tree rooted at top (including top
    itself, but excluding '.' and '..'), yields a 3-tuple
    
        dirpath, dirnames, filenames
    
    dirpath is a string, the path to the directory.  dirnames is a list of
    the names of the subdirectories in dirpath (excluding '.' and '..').
    filenames is a list of the names of the non-directory files in dirpath.
    Note that the names in the lists are just names, with no path components.
    To get a full path (which begins with top) to a file or directory in
    dirpath, do os.path.join(dirpath, name).
    
    If optional arg 'topdown' is true or not specified, the triple for a
    directory is generated before the triples for any of its subdirectories
    (directories are generated top down).  If topdown is false, the triple
    for a directory is generated after the 

In [22]:
def get_imports(path=Path.cwd(), recursive=True):
    """ Return a `set` containing the names of the modules imported by `path`.
    """
    if type(path) is str: path = Path(path)
    FILES = map(lambda p: p.stem, map(lambda s: Path(s), glob(str(path / "*.py"), recursive=recursive)))
    pp(FILES)
#     LOCAL_MODULES = sorted([n for n in FILES if not n.startswith('.')], key=str.lower)
#     print(f'{LOCAL_MODULES=}')
    results = [set(), set(), set()]
    if type(path) is str:
        path = Path(path)
    if path.is_dir():
        for f in glob(str(path / "*.py")):
            result = get_imports(f)
            results = [r.union(result[i]) for i, r in enumerate(results)]
    else:
        result = set()
        lines = path.read_text().split('\n')
        regex = re.compile("\s*import (\w*)|\s*from (\w*)")
        for s in lines:
            m = regex.match(s)
            if m:
                i = 1
                while not m.group(i):
                    i += 1
                assert i < 4
                word = m.group(i)
                if word:
                    m2 = re.search(r'(\w*)\.\w*', word)
                    if m2:
                        word = m2.group(1)
                    result.add(word)
        for r in result:
            if r in PYTHON_MODULES:
                results[0].add(r)
            elif r in LOCAL_MODULES:
                results[2].add(r)
            else:
                # if not r == "py"
                results[1].add(r)
        results = [sorted(list(r)) for r in results]
    return list(map(lambda s: list(s), results))

def print_imports(path=Path.cwd()):
    results = get_imports(path)
    if len(results[0]):
        print("Python Modules:")
        print()
        columnize(results[0])
        print()
    if len(results[1]):
        print("Packages:")
        print()
        columnize(results[1])
        print()
    if len(results[2]):
        print("Local Modules:")
        print()
        columnize(results[2])
        print()


In [23]:
# print_imports('../hw')

In [24]:
path = BASEDIR
path.name

'hw-4.2.0'

In [25]:
str(path)

'/home/fuzzy/Development/hw-4.2.0'

In [26]:
!pwd

/home/fuzzy/Projects/develop-4.2.0/hw-4.2.0/nb


In [27]:
def ignore(name:str):
    """Ignore directories that start with '_' and files that start with '.' or '_'."""
#     print(name)
    IGNORE = {'.', '_', 'tests'}
    components = name.split(os.sep)
#     pp(components)
    if len(components) > 1:
        for s in name.split(os.sep)[:-1]:
            for i in IGNORE:
                if s.startswith(i): return True
    p = Path(name)
    if p.is_dir() and p.name.startswith('_'): return True
    else:
        for i in IGNORE:
            if p.name.startswith(i) and not p.stem in {'__init__', '__main__'}:
                return True
    return False

In [28]:
df = DataFrame([f for f in os.walk(path, followlinks=True) if not ignore(f[0])], columns=['dir', 'subdirs', 'files'])

In [29]:
df

,dir,subdirs,files
0,/home/fuzzy/Development/hw-4.2.0,"[tests, etc, sql, md, html, .git, nb, data, .v...","[run, .gitignore, TODO.md, LICENSE, README.md,..."
1,/home/fuzzy/Development/hw-4.2.0/etc,[],"[config.ini, setup.ini, arguments.json]"
2,/home/fuzzy/Development/hw-4.2.0/sql,[],"[clear.sql, now.sql]"
3,/home/fuzzy/Development/hw-4.2.0/md,[.ipynb_checkpoints],[desc.md]
4,/home/fuzzy/Development/hw-4.2.0/html,"[js, css]","[index.php, all the kings horses.txt, favicon...."
5,/home/fuzzy/Development/hw-4.2.0/html/js,[.ipynb_checkpoints],[scripts.js]
6,/home/fuzzy/Development/hw-4.2.0/html/css,[],[default.css]
7,/home/fuzzy/Development/hw-4.2.0/nb,[.ipynb_checkpoints],"[program.ipynb, sftp.ipynb, args.ipynb, links...."
8,/home/fuzzy/Development/hw-4.2.0/data,[],"[epilog.txt, deweysummaries.pdf, deweysummarie..."
9,/home/fuzzy/Development/hw-4.2.0/sh,[.ipynb_checkpoints],"[activate.sh, hw.sh, library.sh, bashrc]"


In [30]:
type(df['files'])

pandas.core.series.Series

In [31]:
help(os.walk)

Help on function walk in module os:

walk(top, topdown=True, onerror=None, followlinks=False)
    Directory tree generator.
    
    For each directory in the directory tree rooted at top (including top
    itself, but excluding '.' and '..'), yields a 3-tuple
    
        dirpath, dirnames, filenames
    
    dirpath is a string, the path to the directory.  dirnames is a list of
    the names of the subdirectories in dirpath (excluding '.' and '..').
    filenames is a list of the names of the non-directory files in dirpath.
    Note that the names in the lists are just names, with no path components.
    To get a full path (which begins with top) to a file or directory in
    dirpath, do os.path.join(dirpath, name).
    
    If optional arg 'topdown' is true or not specified, the triple for a
    directory is generated before the triples for any of its subdirectories
    (directories are generated top down).  If topdown is false, the triple
    for a directory is generated after the 

In [32]:
public(Series)

T                dropna                   map             searchsorted  var  
abs              dt                       mask            sem           view 
add              dtype                    max             set_axis      where
add_prefix       dtypes                   mean            set_flags     xs   
add_suffix       duplicated               median          shape       
agg              empty                    memory_usage    shift       
aggregate        eq                       min             size        
align            equals                   mod             skew        
all              ewm                      mode            slice_shift 
any              expanding                mul             sort_index  
append           explode                  multiply        sort_values 
apply            factorize                name            sparse      
argmax           ffill                    nbytes          squeeze     
argmin           fillna                   ndim   

In [33]:
L = list()
for i, l in enumerate(df['files']):
    L.extend([Path(df['dir'][i])/s for s in l])
L = [p for p in L if not ignore(str(p)) and p.suffix in {'.py', '.ipynb'}]

In [34]:
notebooks = Series(list(filter(lambda p: p.name.endswith('.ipynb'), L)), dtype=object).unique()
sources = Series(list(filter(lambda p: p.name.endswith('.py'), L)), dtype=object).unique()

In [35]:
pp(notebooks)

array([PosixPath('/home/fuzzy/Development/hw-4.2.0/nb/program.ipynb'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/nb/sftp.ipynb'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/nb/args.ipynb'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/nb/links.ipynb'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/nb/TODO.ipynb'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/nb/hw.ipynb'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/nb/sass.ipynb'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/nb/profiler.ipynb'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/nb/bash.ipynb'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/nb/pdf.ipynb'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/nb/xdg.ipynb'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/nb/lint.ipynb'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/nb/git.ipynb'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/nb/worksheet.ipynb'),
       PosixPath('/home/fuzzy/Developmen

In [36]:
with notebooks[0].open() as f:
    j = json.load(f)

In [37]:
pat = re.compile(r'^(from|import)')
lines = list()
for c in j['cells']:
    lines.extend([l.rstrip('\n') for l in c['source'] if pat.match(l)])
lines

['from pathlib import Path',
 'import site',
 'from tools import *',
 'from program import Program']

In [38]:
pat = re.compile(r'^(from|import)')
lines = list()
for nb in notebooks:
    with nb.open() as f:
        j = json.load(f)
        for c in j['cells']:
            lines.extend([l.rstrip('\n') for l in c['source'] if pat.match(l)])
lines

['from pathlib import Path',
 'import site',
 'from tools import *',
 'from program import Program',
 'import ftplib',
 'from ftplib import FTP',
 'from io import StringIO',
 'import os',
 'from pathlib import Path',
 'import site',
 'import sys',
 'from tools import *',
 'from program import Program',
 'from IPython.display import Markdown',
 'import json',
 'import os',
 'from pathlib import Path',
 'import site',
 'import subprocess',
 'from warnings import warn',
 'from bs4 import BeautifulSoup',
 'import isbnlib',
 'import mysql.connector',
 'import pdfminer',
 'import requests',
 'from tools import *',
 'from program import Program',
 'from functools import partial',
 'from io import StringIO',
 'from os import chdir, listdir',
 'import pkgutil',
 'import xdg',
 'from hw.tools import *',
 'import xdg.util',
 'import xdg.BaseDirectory',
 'from os import chdir',
 'from pathlib import Path',
 'import sys',
 'from py.tools import * # python helper functions for this notebook',
 'impo

In [46]:
lines.extend(grep(*sources, pattern=pat, quiet=True))
print(len(lines))
s = set(lines)
print(len(s))

Debugging `grep().
type(pattern)=<class 're.Pattern'>
Processing pattern re.compile('^(from|import)')
525
85


In [52]:
modules = [line.split()[1] for line in s]
modules

['warnings',
 'mysql.connector',
 'functools',
 'isbnlib',
 'glob',
 'xdg',
 'tools',
 'ftplib',
 'tempfile',
 'os',
 'os',
 'py.tools',
 'pdfminer',
 'hw.hw',
 'enum',
 'markdown',
 'sys',
 'configparser',
 'os.path',
 'inspect',
 'ftplib',
 'logging',
 'subprocess',
 'os',
 'pkg_resources',
 'os',
 'datetime',
 'IPython.display',
 'constants',
 'requests',
 'tarfile',
 'tools',
 'functools',
 'os',
 'datetime',
 'glob',
 'pandas',
 'shutil',
 'pdb',
 'walkdir',
 'pkgutil',
 'tokenize',
 'xdg.BaseDirectory',
 'fileinput',
 'hw.tools',
 'xdg.util',
 'os',
 'program',
 'file',
 'pdb',
 'glob',
 'pathlib',
 'importlib.resources',
 'shlex',
 're',
 'pathlib',
 'netsnmp',
 'os',
 'subprocess',
 'json',
 'os',
 'traceback',
 'os',
 'pprint',
 'getpass',
 'os',
 'shutil',
 'driver',
 'tools.tools',
 'bs4',
 'functools',
 'warnings',
 'collections',
 'xdg.BaseDirectory',
 'site',
 'zipfile',
 'os',
 'gzip',
 'shutil',
 'io',
 'cmd',
 'xdg.BaseDirectory',
 'pandas',
 'pandas',
 'argparse']

In [49]:
public(s)

add         difference_update    isdisjoint  remove                       update
clear       discard              issubset    symmetric_difference       
copy        intersection         issuperset  symmetric_difference_update
difference  intersection_update  pop         union                      


In [50]:
s.intersection(PYTHON_MODULES)

set()

In [55]:
pymods = set(modules).intersection(PYTHON_MODULES)

In [59]:
columnize(sorted(list(pymods), key=str.lower))

argparse      enum       glob     logging  pprint  subprocess  traceback
cmd           fileinput  gzip     os       re      sys         warnings 
collections   ftplib     inspect  pathlib  shlex   tarfile     zipfile  
configparser  functools  io       pdb      shutil  tempfile  
datetime      getpass    json     pkgutil  site    tokenize  


In [110]:
pkgmods = set(modules).difference(PYTHON_MODULES).difference(srcmods)

In [111]:
columnize(sorted(list(pkgmods), key=str.lower))

bs4                  IPython.display  os.path        requests           xdg.util
file                 isbnlib          pandas         tools.tools      
hw.hw                markdown         pdfminer       walkdir          
hw.tools             mysql.connector  pkg_resources  xdg              
importlib.resources  netsnmp          py.tools       xdg.BaseDirectory


In [112]:
sources

array([PosixPath('/home/fuzzy/Development/hw-4.2.0/bin/create.py'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/hw/startup.py'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/hw/program.py'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/hw/hw.py'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/hw/create.py'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/hw/__main__.py'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/hw/imports.py'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/hw/library.py'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/hw/tools.py'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/hw/constants.py'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/hw/setup.py'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/hw/driver.py'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/hw/__init__.py'),
       PosixPath('/home/fuzzy/Development/hw-4.2.0/hw/files.py')],
      dtype=object)

In [65]:
src_names = set([p.name.split('.')[0] for p in sources])

In [73]:
srcmods = src_names.intersection(modules)

In [68]:
s

{'from IPython.display import Markdown',
 'from argparse import ArgumentParser',
 'from bs4 import BeautifulSoup',
 'from cmd import Cmd',
 'from collections import ChainMap',
 'from configparser import ConfigParser',
 'from constants import *',
 'from datetime import date',
 'from datetime import datetime as dt',
 'from driver import Driver',
 'from enum import IntEnum',
 'from file import Magic',
 'from ftplib import FTP',
 'from functools import partial',
 'from functools import partial, singledispatch, wraps',
 'from functools import singledispatch, singledispatchmethod',
 'from getpass import getuser',
 'from glob import glob',
 'from glob import iglob as glob',
 'from hw.hw import HelloWorld',
 'from hw.tools import *',
 'from io import StringIO',
 'from markdown import markdown as md',
 'from os import chdir',
 'from os import chdir as cd',
 'from os import chdir as cd, curdir as pwd, listdir as ls',
 'from os import chdir as cd, curdir, listdir as ls',
 'from os import chdir as

In [117]:
output = list()
for mods in [pymods, pkgmods, srcmods]:
    section = list()
    for line in s:
        word = line.split()[1]
        if word in mods:
            section.append(line)
#     pp(section)
#     print()
    output.extend(sorted(section, key=lambda s: s.split()[1].lower()))
    output.append('')
output

['from argparse import ArgumentParser',
 'from cmd import Cmd',
 'from collections import ChainMap',
 'from configparser import ConfigParser',
 'from datetime import date',
 'from datetime import datetime as dt',
 'from enum import IntEnum',
 'import fileinput',
 'import ftplib',
 'from ftplib import FTP',
 'from functools import partial, singledispatch, wraps',
 'from functools import partial',
 'from functools import singledispatch, singledispatchmethod',
 'from getpass import getuser',
 'from glob import glob',
 'from glob import iglob as glob',
 'import glob',
 'import gzip',
 'import inspect',
 'from io import StringIO',
 'import json',
 'import logging',
 'import os',
 'from os import chdir as cd',
 'from os import chdir, listdir',
 'from os import environ',
 'from os import chdir as cd, curdir, listdir as ls',
 'from os import chdir as cd, listdir',
 'from os import chdir as cd, curdir as pwd, listdir as ls',
 'from os import listdir',
 'from os import chdir, curdir, listdir',
 

In [121]:
print(Path('hw/imports.py').read_text())

from argparse import ArgumentParser
from cmd import Cmd
from collections import ChainMap
from configparser import ConfigParser
from datetime import date
from datetime import datetime as dt
from enum import IntEnum
import fileinput
import ftplib
from ftplib import FTP
from functools import partial, singledispatch, wraps
from functools import partial
from functools import singledispatch, singledispatchmethod
from getpass import getuser
from glob import glob
from glob import iglob as glob
import glob
import gzip
import inspect
from io import StringIO
import json
import logging
import os
from os import chdir as cd
from os import chdir, listdir
from os import environ
from os import chdir as cd, curdir, listdir as ls
from os import chdir as cd, listdir
from os import chdir as cd, curdir as pwd, listdir as ls
from os import listdir
from os import chdir, curdir, listdir
from os import chdir as cd, listdir as ls
from os import chdir
from pathlib import Path, PosixPath, WindowsPath
from pathlib 

In [120]:
Path('hw/imports.py').write_text('\n'.join(output))

2299

In [81]:
public(DataFrame)

T               drop_duplicates    last              resample       to_parquet  
abs             droplevel          last_valid_index  reset_index    to_period   
add             dropna             le                rfloordiv      to_pickle   
add_prefix      dtypes             loc               rmod           to_records  
add_suffix      duplicated         lookup            rmul           to_sql      
agg             empty              lt                rolling        to_stata    
aggregate       eq                 mad               round          to_string   
align           equals             mask              rpow           to_timestamp
all             eval               max               rsub           to_xarray   
any             ewm                mean              rtruediv       to_xml      
append          expanding          median            sample         transform   
apply           explode            melt              select_dtypes  transpose   
applymap        ffill       

In [83]:
df.to_string()

'        0     1       2      3     4     5\n0    from  glob  import   glob  None  None\n1    from  glob  import  iglob    as  glob\n2  import  glob    None   None  None  None'

In [84]:
print(df.to_string())

        0     1       2      3     4     5
0    from  glob  import   glob  None  None
1    from  glob  import  iglob    as  glob
2  import  glob    None   None  None  None
